In [2]:
import os
os.chdir("/home/data")

In [11]:
!csvcut -n flights.csv

  1: Year
  2: Month
  3: DayofMonth
  4: DayOfWeek
  5: DepTime
  6: CRSDepTime
  7: ArrTime
  8: CRSArrTime
  9: UniqueCarrier
 10: FlightNum
 11: TailNum
 12: ActualElapsedTime
 13: CRSElapsedTime
 14: AirTime
 15: ArrDelay
 16: DepDelay
 17: Origin
 18: Dest
 19: Distance
 20: TaxiIn
 21: TaxiOut
 22: Cancelled
 23: CancellationCode
 24: Diverted
 25: CarrierDelay
 26: WeatherDelay
 27: NASDelay
 28: SecurityDelay
 29: LateAircraftDelay


In [63]:
!head -n 1000 flights.csv | awk 'BEGIN { FS=","; OFS="\t"; } NR>1 {print $9,$10,$15}' | head

29	WN	2891	1
29	WN	462	8
29	WN	1229	34
29	WN	1355	26
29	WN	2278	-3
29	WN	2386	3
29	WN	409	47
29	WN	1131	-2
29	WN	1212	44
29	WN	2456	-7
